# RECOMMENDATION ENGINE:

### Notebook Contents:

- Content-Based Filtering (and Popularity Based Recommendation) using TMDB 5000 Dataset
- Collaborative Filtering using MovieLens Dataset
- Evaluation

In [1]:
import pandas as pd
import numpy as np

# 1. Content-Based Filtering

### - Importing the Datasets.

In [2]:
movies=pd.read_csv('dataset/tmdb_5000_movies.csv')
credits=pd.read_csv('dataset/tmdb_5000_credits.csv')
links=pd.read_csv('dataset/links.csv')

### - Preprocessing the Imported Data

Merging tmdb_5000_movies.csv and tmdb_5000_credits.csv on the basis of title column.

In [3]:
movies=movies.merge(credits, on='title')
movies=movies.merge(links, on='id')

We keep the relevant columns and remove the rest.

In [4]:
movies=movies[['id','title','overview','genres','keywords','cast','crew','popularity','vote_average','movieId']]

Converting genres and keywords fields to list format to make processing easier.

In [5]:
import ast

In [6]:
def convert1(item):
    arr=[]
    for i in ast.literal_eval(item):
        arr.append(i['name'])
    return arr

movies['genres']=movies['genres'].apply(convert1)
movies['keywords']=movies['keywords'].apply(convert1)

Converting cast field to list format and reducing data to the name of the first 5 most significant cast members, to make processing easier.

In [7]:
def convert2(item):
    arr=[]
    count=0
    for i in ast.literal_eval(item):
        if count!=5:
            arr.append(i['name'])
            count+=1
        else:
            break
    return arr

movies['cast']=movies['cast'].apply(convert2)

Extracting Director's details from crew field and rejecting other insignificant information.

In [8]:
def convert3(obj):
    L=[]
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
            L.append(i['name'])
            break
    return L

movies['crew']=movies['crew'].apply(convert3)

Removing white spaces from keywords, cast and crew fields to avoid discrepancies.

In [9]:
movies['keywords']=movies['keywords'].apply(lambda x: [i.replace(" ","") for i in x])
movies['cast']=movies['cast'].apply(lambda x: [i.replace(" ","") for i in x])
movies['crew']=movies['crew'].apply(lambda x: [i.replace(" ","") for i in x])

Splitting the string in overview field to a list of strings.

In [10]:
movies['overview']=movies['overview'].apply(lambda x: str(x).split())

Incorporating the overview, genres, keywords, cast and crew fields into a single tags fields.

In [11]:
movies['tags']=movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']
new_df=movies[['id','title','tags','popularity','vote_average','movieId']]

Joining the list of strings in tags field into one single string that can be used for model training.

In [12]:
new_df['tags']=new_df['tags'].apply(lambda x: " ".join(x))
new_df['tags']=new_df['tags'].apply(lambda x: x.lower())

C:\Users\DELL\AppData\Local\Temp\ipykernel_13600\2380013068.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x: " ".join(x))
C:\Users\DELL\AppData\Local\Temp\ipykernel_13600\2380013068.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x: x.lower())


### - Model Building

Text Vectorisation - Using the Natural Language Toolkit, we reduce all words to stem words. This is necessary to ensure that similar words are processed by the model as one entity. 

In [13]:
import nltk
from nltk.stem.porter import PorterStemmer

ps=PorterStemmer()

def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

new_df['tags']=new_df['tags'].apply(stem)

C:\Users\DELL\AppData\Local\Temp\ipykernel_13600\2139505822.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(stem)


CountVectorizer class from sklearn to vectorise the tags. Vector Similarity Matrix using cosine_similarity as a similarity measure between Movies.

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

cv=CountVectorizer(max_features=5000, stop_words='english')
vectors=cv.fit_transform(new_df['tags']).toarray()

movie_similarity=cosine_similarity(vectors)

Creating a function recommend which takes a movie name as input and returns the top 5 closest vectors (similar movies) to it. This will be our blueprint for all recommend functions in the future.

In [15]:
def recommend(movie):
    movie_index=(new_df[new_df['title']==movie].index[0])
    distances=movie_similarity[movie_index]
    movies_list=sorted(list(enumerate(distances)), reverse=True, key=lambda x:x[1])[1:6]

    for i in movies_list:
        print(new_df.iloc[i[0]].title)

Using pickle to create pickle files and storing our created Similarity Matrix. Our application will be able to derive closest vectors during recommendation from these pickle files.

In [16]:
import pickle

pickle.dump(new_df.to_dict(),open('movies.pkl','wb'))
pickle.dump(movie_similarity,open('recommend_1.pkl','wb'))

# 2. User-Based Collaborative Filtering

Importing the user ratings dataset.

In [17]:
ratings = pd.read_csv('dataset/ratings.csv')

Splitting the dataset into training data and testing data, and training the model.

In [18]:
from sklearn.model_selection import train_test_split

x_train, x_test = train_test_split(ratings, test_size = 0.30, random_state = 42)
data = x_train.pivot(index = 'userId', columns = 'movieId', values = 'rating').fillna(0)

Making a copy of train and test datasets. 

These dummy datasets will be used to check whether user has given rating. The movies not rated by user is marked as 1 for prediction. The movies not rated by user is marked as 0 for evaluation.

In [19]:
dummy_train = x_train.copy()
dummy_test = x_test.copy()

dummy_train['rating'] = dummy_train['rating'].apply(lambda x: 0 if x > 0 else 1)
dummy_test['rating'] = dummy_test['rating'].apply(lambda x: 1 if x > 0 else 0)

In [20]:
dummy_train = dummy_train.pivot(index = 'userId', columns = 'movieId', values = 'rating').fillna(1)
dummy_test = dummy_test.pivot(index ='userId', columns = 'movieId', values = 'rating').fillna(0)

User Similarity Matrix using Cosine similarity as a similarity measure between Users.

In [21]:
from sklearn.metrics.pairwise import cosine_similarity

user_similarity = cosine_similarity(data)
user_similarity[np.isnan(user_similarity)] = 0

We do not want to recommend the same movie that the user already watched. We will ignore the movies rated by the user and we will use our dummy training matrix. 

In [22]:
user_predicted_ratings = np.dot(user_similarity, data)
user_final_ratings = np.multiply(user_predicted_ratings, dummy_train)

Using pickle to create pickle files and storing our created User Similarity Matrix. Our application will be able to derive closest vectors during recommendation from these pickle files.

In [23]:
pickle.dump(user_final_ratings,open('recommend_2.pkl','wb'))

# 3. Item-Based Collaborative Filtering

In [24]:
movie_features = x_train.pivot(index = 'movieId', columns = 'userId', values = 'rating').fillna(0)

In [25]:
from sklearn.metrics.pairwise import cosine_similarity

item_similarity = cosine_similarity(movie_features)
item_similarity[np.isnan(item_similarity)] = 0

In [26]:
item_predicted_ratings = np.dot(movie_features.T, item_similarity)
item_final_ratings = np.multiply(item_predicted_ratings, dummy_train)

In [53]:
item_final_ratings.iloc[42:43].sort_values(by=43,ascending = False)[0:5]

movieId,1,2,3,4,5,6,7,8,9,10,...,190221,191005,193565,193571,193573,193579,193581,193583,193585,193609
userId,,,,,,,,,,,,,,,,,,,,,
43,67.074925,78.342161,51.056146,32.737629,0.0,45.10717,46.727041,0.0,17.888433,0.0,...,0.207754,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.354246


# 3. Evaluation

### For User-Based Collaborative Filtering

We will evaluate for the movies already rated by the User.

In [28]:
test_user_features = x_test.pivot(index = 'userId', columns = 'movieId', values = 'rating').fillna(0)
test_user_similarity = cosine_similarity(test_user_features)
test_user_similarity[np.isnan(test_user_similarity)] = 0

In [29]:
user_predicted_ratings_test = np.dot(test_user_similarity, test_user_features)
test_user_final_rating = np.multiply(user_predicted_ratings_test, dummy_test)

Normalize the final ratings in range 0.5 to 5

In [30]:
from sklearn.preprocessing import MinMaxScaler

X = test_user_final_rating.copy() 
X = X[X > 0]
scaler = MinMaxScaler(feature_range = (0.5, 5))
scaler.fit(X)
pred = scaler.transform(X)

total_non_nan = np.count_nonzero(~np.isnan(pred))

In [31]:
test = x_test.pivot(index = 'userId', columns = 'movieId', values = 'rating')

Finding the Root Mean Square Error (RMSE)

In [32]:
diff_sqr_matrix = (test - pred)**2
sum_of_squares_err = diff_sqr_matrix.sum().sum() 

rmse = np.sqrt(sum_of_squares_err/total_non_nan)
print(rmse)

1.5635654266606624


Finding the Mean Absolute Error

In [33]:
mae = np.abs(pred - test).sum().sum()/total_non_nan
print(mae)

1.2116342196650216


### For Item-Based Collaborative Filtering

In [54]:
test_item_features = x_test.pivot(index = 'movieId', columns = 'userId', values = 'rating').fillna(0)
test_item_similarity = cosine_similarity(test_item_features)
test_item_similarity[np.isnan(test_item_similarity)] = 0 

In [55]:
item_predicted_ratings_test = np.dot(test_item_features.T, test_item_similarity )
test_item_final_rating = np.multiply(item_predicted_ratings_test, dummy_test)

In [56]:
from sklearn.preprocessing import MinMaxScaler

X = test_item_final_rating.copy() 
X = X[X > 0]

scaler = MinMaxScaler(feature_range = (0.5, 5))
scaler.fit(X)
pred = scaler.transform(X)

In [57]:
total_non_nan = np.count_nonzero(~np.isnan(pred))
test = x_test.pivot(index = 'userId', columns = 'movieId', values = 'rating')

In [58]:
diff_sqr_matrix = (test - pred)**2
sum_of_squares_err = diff_sqr_matrix.sum().sum()

rmse = np.sqrt(sum_of_squares_err/total_non_nan)
print(rmse)

2.512699212653213


In [59]:
mae = np.abs(pred - test).sum().sum()/total_non_nan
print(mae)

2.215407217950911


From the RMSE and MAE values, we can conclude that User-Based Collaborative Filtering is more efficient than Item-Based Collaborative Filtering. Hence we have implemented User-Based Collaborative Filtering in teh web application.